In [94]:
import pandas as pd
import nibabel as nib
import matplotlib.pylab as plt
import numpy as np
import os  # needed to navigate the system to get the input data
import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics
from glob import glob
import SimpleITK as sitk
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape, ngtdm, gldm
import tqdm
from collections import defaultdict
import scipy.integrate as integrate
import scipy
import scipy.ndimage
import csv


In [95]:
import os

CT=[]
pt_list=[]
data_path = '/media/ericyang/DATA/stroke_CT'
for pt in os.listdir(data_path):
    #print(pt)
    for ser in os.listdir(os.path.join(data_path, pt)):
        if ser== 'CT.nii.gz':
            path = os.path.join(data_path, pt, ser)
            CT.append(path)
            pt_list.append(pt)
            #print(os.path.join(data_path, pt,'CT', ser))

print(len(CT))
print(len(pt_list))

196
196


In [96]:
df = pd.read_csv('/home/nidhiramesh/CT_Project/data_sheets/MTB_CT_Patients.csv')
df.head()


,PT_ID,Sex,Age,Side,Location,NIHSS,TICI,AOL,Pulls,Label_mTICI_2c+,Label_FPE_2c,Label_mTICI_2b+,Label_FPE_2b
0,280019,m,67,R,MCA,11,3,3,4,1,0,1,0
1,280021,m,66,R,MCA,10,0,0,3,0,0,0,0
2,280022,f,73,R,ICA,23,2b,3,3,0,0,1,0
3,280025,f,83,L,ICA,19,2b,3,2,0,0,1,0
4,280037,m,59,B,PICA,32,3,3,1,1,1,1,1


In [97]:
df = df.loc[df['PT_ID'].isin(pt_list)]

In [98]:
df.shape

(186, 13)

In [99]:
# new_pt = []
# for x in pt_list:
#     for y in df2['PT_ID']:
#         if x==y:
#             new_pt.append(x)
    
        
# print(len(new_pt))

In [100]:
location = df['Location'].tolist()
side = df['Side'].tolist()

location = [x.strip(' ') for x in location]
print(len(location))

spot = []
for l in location:
    s= side[location.index(l)]
    spot.append(s+'_'+l)
print(spot)

df['spot']=spot



186
['R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'B_PICA', 'L_ICA', 'L_ICA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'B_PCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'B_PCA', 'B_PCA', 'L_ICA', 'R_MCA', 'R_MCA', 'L_ICA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ACA', 'L_ICA', 'R_MCA', 'L_ICA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'R_MCA', 'R_MCA', 'B_PCA', 'B_PCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'R_MCA', 'L_ICA', 'L_ICA', 'L_ICA', 'R_MCA', 'R_MCA', 'R_M

In [101]:
#Dropping all patients whose side is 'B'
drop_id=[]
df = df[df.Side != 'B'] 


#Dropping patients whose location is CCA (count=3)
drop_id.append(df['PT_ID'].isin(df.Side != 'B'))
df = df[df.Location != 'CCA']
df.shape

mTICI = df['Label_mTICI_2c+']
print(mTICI)

mTICI.to_csv('mTICI_labels.csv',index=False)
df.shape



0      1
1      0
3      0
5      0
9      0
      ..
246    1
247    1
248    0
250    1
251    1
Name: Label_mTICI_2c+, Length: 178, dtype: int64


(178, 14)

In [102]:
# #checking to find any stray locations
# count=0
# for x in spot:
#     #print(x)
#     if x != 'R_ACA'and x != 'R_MCA'and x != 'R_PCA'and x != 'R_ICA'and x != 'R_PICA'and x != 'L_ACA'and x != 'L_MCA'and x != 'L_PCA'and x != 'L_ICA'and x != 'L_PICA':
#         count +=1
#         print(x)
#         spot.remove(x)
#         df = df[df.Location != x]
        
        
        
# print(count)

In [103]:
print(len(spot))

186


In [104]:
# new_pt = []
# for x in pt_list:
#     for y in df['PT_ID']:
#         if x==y:
#             new_pt.append(x)
    
        
# print(len(new_pt))


178


In [105]:
df.head()

,PT_ID,Sex,Age,Side,Location,NIHSS,TICI,AOL,Pulls,Label_mTICI_2c+,Label_FPE_2c,Label_mTICI_2b+,Label_FPE_2b,spot
0,280019,m,67,R,MCA,11,3,3,4,1,0,1,0,R_MCA
1,280021,m,66,R,MCA,10,0,0,3,0,0,0,0,R_MCA
3,280025,f,83,L,ICA,19,2b,3,2,0,0,1,0,L_ICA
5,280043,m,86,R,MCA,27,2b,3,3,0,0,1,0,R_MCA
9,280079,m,50,R,MCA,20,2b,3,1,0,0,1,1,R_MCA


In [106]:
# new_CT=[]
# for pt in new_pt:
#     #print(pt)
#     for ser in os.listdir(os.path.join(data_path, pt)):
#         if ser== 'CT.nii.gz':
#             path = os.path.join(data_path, pt, ser)
#             new_CT.append(path)
#             #pt_list.append(pt)
#             #print(os.path.join(data_path, pt,'CT', ser))


In [107]:
print(len(new_CT))

178


In [110]:
#checking to find any stray locations


count=0
for x in spot:
    #print(x)
    if x != 'R_ACA'and x != 'R_MCA'and x != 'R_PCA'and x != 'R_ICA'and x != 'R_PICA'and x != 'L_ACA'and x != 'L_MCA'and x != 'L_PCA'and x != 'L_ICA'and x != 'L_PICA':
        count +=1
        df=df[df['spot'] != x]
        spot.remove(x)
       
        #df=df[df.spot!=x]
       
        
        
        
print(len(spot))
print(len(df['spot']))

177
175


In [111]:
new_pt = []
for x in pt_list:
    for y in df['PT_ID']:
        if x==y:
            new_pt.append(x)
    
        
print(len(new_pt))


175


In [112]:
new_CT=[]
for pt in new_pt:
    #print(pt)
    for ser in os.listdir(os.path.join(data_path, pt)):
        if ser== 'CT.nii.gz':
            path = os.path.join(data_path, pt, ser)
            new_CT.append(path)
            #pt_list.append(pt)
            #print(os.path.join(data_path, pt,'CT', ser))
# ?

In [113]:
img = nib.load('/home/jennifer/Projects/Stroke/Data/stroke_CT/289008/CT.nii.gz')
spacing = img.header['pixdim'][1:4]
print(spacing)

[0.7374631 0.7374631 6.3126845]


In [114]:
print(df['spot'])

0      R_MCA
1      R_MCA
3      L_ICA
5      R_MCA
9      R_MCA
       ...  
246    R_MCA
247    R_MCA
248    R_MCA
250    R_MCA
251    R_MCA
Name: spot, Length: 175, dtype: object


In [115]:
atlas = nib.load('/home/nidhiramesh/CT_Project/vasc_terr_r.nii.gz')
atlas = atlas.get_fdata().astype(int) / 2

label_terr = ['Whole_Brain',
              'R_ACA',
              'R_MCA',
              'R_PCA',
              'R_ICA',
              'R_PICA',
              'L_ACA',
              'L_MCA',
              'L_PCA',
              'L_ICA',
              'L_PICA', ]

all_masks = []
for l in df['spot']:
    ind = l
    #print(ind)
    mask = np.zeros(atlas.T.shape)
    mask += atlas.T == label_terr.index(ind)
    plt.imshow(mask[10,:,:], cmap='gray')
    #plt.show()
    all_masks.append(mask)
 

In [116]:
print(len(all_masks))
print(len(new_CT))

175
175


In [117]:
for x in range(len(new_CT)):
    img = nib.load(new_CT[x])
    mask = all_masks[x]
    
    header = img.header
    spacing = img.header['pixdim'][1:4]
    img = img.get_fdata()
    #img= np.flipud(np.rot90(img))
    # spacing = np.array(all_spacings)
    new_spacing = [0.7374631, 0.7374631, 6.3126845]
    resize_factor = spacing / new_spacing
    new_real_shape = img.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / img.shape
    img = scipy.ndimage.zoom(img, real_resize_factor, order=3)
    new_CT[x] = img

    #mask = mask.get_fdata()
    #mask= np.flip(new_CT[x].shape)
    mask= np.rot90(mask,2,axes=(1,2))
    mask= np.fliplr(mask)
    mask= mask.T
    #mask = np.rot90(mask)
     # spacing = np.array(all_spacings)
    new_spacing = [0.7374631, 0.7374631, 6.3126845]
    resize_factor = spacing / new_spacing
    new_real_shape = mask.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / mask.shape
    img_mask = scipy.ndimage.zoom(mask, real_resize_factor, order=0)
    all_masks[x]= img_mask

In [118]:
print(new_CT[21].shape)
print(all_masks[21].shape)

(207, 256, 26)
(207, 256, 26)


In [119]:
radiomic_features_CT = []
for n, m in zip(new_CT, all_masks):
    new_img =sitk.GetImageFromArray(n) # 'put your t2 numpy array'
    new_img.SetSpacing((0.7374631, 0.7374631, 6.3126845))
    #new_img.SetSpacing((0.5, 0.5, 3))
     
    new_mask = sitk.GetImageFromArray(m) # 'put your prostate mask numpy array' 
    new_mask.SetSpacing((0.7374631, 0.7374631, 6.3126845))
    extractor = featureextractor.RadiomicsFeatureExtractor()                    
   
    extractor.enableAllFeatures()

    extractor.enableImageTypeByName('LoG',True,  {'sigma': [2, 3, 4, 5]})
    extractor.enableImageTypeByName('Wavelet', True)
    features_mask_CT = extractor.execute(new_img, new_mask)
    radiomic_features_CT.append(features_mask_CT)
        


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calc

In [121]:
df = pd.DataFrame.from_dict(radiomic_features_CT)
df.insert(0, "Patient_ID", new_pt , True)
  
df.head()
#rows are the patients 
#columns are the radiomic feature name 

,Patient_ID,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,...,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
0,280019,v3.0.1,1.21.2,2.1.1,1.2.0,3.7.11,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [2, 3, 4, 5]...",e8a0dc302422570124f4c321019c489a86f9da0f,3D,...,125.75033465468309,0.009605878180725929,6.256881624512802,0.03491726974208563,692338.6638790765,9.183308420454173,8.871915313213333e-05,2556.899201123798,0.0009026586312228157,4.531687367338661
1,280021,v3.0.1,1.21.2,2.1.1,1.2.0,3.7.11,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [2, 3, 4, 5]...",8a992487aa0e5066c0f6c7bbb392280aa564eed0,3D,...,667.5106070410233,0.0019379645512018712,7.066827276385065,0.04786877929033698,456244.2660525178,3.7385420650084575,9.044122869085819e-05,4081.5316111996017,0.005164079403237039,3.1729633608067096
2,280025,v3.0.1,1.21.2,2.1.1,1.2.0,3.7.11,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [2, 3, 4, 5]...",75c32624bf6877af8c7daf0491049fc1e618c560,3D,...,124.00606812750834,0.01318182452669367,5.717053681256715,0.07207652977074056,18625.812535018773,1.0109155856456273,0.0015473312806584818,714.6715554084335,0.0037913262171215345,11.998239644520973
3,280043,v3.0.1,1.21.2,2.1.1,1.2.0,3.7.11,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [2, 3, 4, 5]...",f3001f675ae48dc94f48fdbc3616467bab4415fb,3D,...,2267.0787891819796,0.0010220221710601155,7.735042462649418,0.0664233111231549,212240.90506723465,1.9673849511031274,9.623045484395051e-05,12261.777001323115,0.024049657957085843,5.493050172461749
4,280079,v3.0.1,1.21.2,2.1.1,1.2.0,3.7.11,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [2, 3, 4, 5]...",9c0abbd33249ca611a78a206ee426322ddb25120,3D,...,409.05945442588325,0.0019136186401671612,6.688605401654139,0.045294417700130714,339975.7025555317,2.5796700160468418,0.0001025810432841125,5961.907848408309,0.001290556822435536,6.623056265568178


In [123]:
df.to_csv('radiomic_features_CT.csv',index=False)